<a href="https://colab.research.google.com/github/adrianmoses/hate-speech-detection/blob/main/HateSpeechDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentencepiece

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 47.5 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 3.3 MB 36.0 MB/s 
     |████████████████████████████████| 895 kB 13.7 MB/s 
     |████████████████████████████████| 596 kB 34.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install git+https://github.com/PytorchLightning/pytorch-lightning.git@master --upgrade

  Cloning https://github.com/PytorchLightning/pytorch-lightning.git (to revision master) to /tmp/pip-req-build-6pydqyua
  Running command git clone -q https://github.com/PytorchLightning/pytorch-lightning.git /tmp/pip-req-build-6pydqyua
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 125 kB 5.4 MB/s 
     |████████████████████████████████| 329 kB 38.8 MB/s 
     |████████████████████████████████| 829 kB 40.2 MB/s 
     |████████████████████████████████| 1.1 MB 34.0 MB/s 
     |████████████████████████████████| 160 kB 46.2 MB/s 
     |████████████████████████████████| 271 kB 45.0 MB/s 
     |████████████████████████████████| 192 kB 46.7 MB/s 
  Created wheel for pytorch-lightning: filename=pytorch_lightning-1.6.0.dev0-py3-none-any.whl size=1033241 sha256=b3029784e78b5161775612797bd94c624c3c8b240a02830f5c16a1639

In [3]:
# imports

import pandas as pd
import re
import sklearn
from sklearn.model_selection import train_test_split
import transformers
import torch
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
df = pd.read_csv('gdrive/MyDrive/Khilnani_LP_hate_speech_data.csv')

In [6]:
df.head()

,Unnamed: 0,tweet,class
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,1,""" momma said no pussy cats inside my doghouse """,0
2,2,"""@Addicted2Guys: -SimplyAddictedToGuys http://...",0
3,3,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woo...",0
4,4,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these...",0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5593 entries, 0 to 5592
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5593 non-null   int64 
 1   tweet       5593 non-null   object
 2   class       5593 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 131.2+ KB


In [9]:
# remove usernames and mentions
def clean_tweet(tweet):
    return re.sub(r'@\w+:?', "", tweet, flags=re.IGNORECASE)

df.tweet = df.tweet.apply(clean_tweet)

In [10]:
df.head()

,Unnamed: 0,tweet,class
0,0,!!! RT As a woman you shouldn't complain abou...,0
1,1,""" momma said no pussy cats inside my doghouse """,0
2,2,""" -SimplyAddictedToGuys http://t.co/1jL4hi8ZMF...",0
3,3,""" http://t.co/3gzUpfuMev"" woof woof and hot soles",0
4,4,""" Lemmie eat a Oreo &amp; do these dishes."" On...",0


In [11]:
# split text with sklearn
y = df['class']
X = df['tweet']

x_train, x_test, y_train, y_test = train_test_split(X.tolist(), y.tolist())

In [12]:
from transformers import AlbertTokenizer
  
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [13]:
device = 'cuda'
encoded_input = tokenizer(x_train, padding=True, truncation=True, max_length=64, return_tensors='pt')
train_input = [encoded_input['input_ids'].to(device), encoded_input['attention_mask'].to(device),
               encoded_input['token_type_ids'].to(device), torch.tensor(y_train).to(device)]

In [15]:
test_encoded_input = tokenizer(x_test, padding=True, truncation=True, max_length=64, return_tensors='pt')
val_input = [test_encoded_input['input_ids'].to(device), test_encoded_input['attention_mask'].to(device),
               test_encoded_input['token_type_ids'].to(device), torch.tensor(y_test).to(device)]

In [25]:
class TweetDataLoader(pl.LightningDataModule):

    def __init__(self, trn, val, batch_size=32):
        self.batch_size = batch_size
        self.trn = DataLoader(TensorDataset(*trn), batch_size=batch_size)
        self.val = DataLoader(TensorDataset(*val), batch_size=batch_size)

    def train_dataloader(self):
        return self.trn
    
    def val_dataloader(self):
        return self.val


In [26]:
dls = TweetDataLoader(train_input, val_input, batch_size=32)

In [27]:
len(next(iter(dls.trn)))

4

In [28]:
from transformers import AlbertModel

model = AlbertModel.from_pretrained('albert-base-v2')


Downloading:   0%|          | 0.00/45.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
from torch.nn import functional as F

class HateSpeechClassifier(pl.LightningModule):

    def __init__(self, model):
        super().__init__()
        self.model = model
        self.l1 = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.5)
        self.l2 = torch.nn.Linear(768, 2)

        

    def forward(self, x):
        input_ids, attention_mask, token_type_ids = x
        x = self.model(input_ids, 
                               attention_mask=attention_mask, 
                               token_type_ids=token_type_ids)[0]
        x = x[:, 0]
        x = self.dropout(torch.relu(self.l1(x)))
        return torch.log_softmax(self.l2(x), dim=1)


    def training_step(self, batch, batch_idx):
        input_ids, attention_masks, token_type_ids, y = batch
        y_hat = self([input_ids, attention_masks, token_type_ids])
        loss = F.nll_loss(y_hat, y.view(-1))
        return loss


    def validation_step(self, batch, batch_idx):
        input_ids, attention_masks, token_type_ids, y = batch
        y_hat = self([input_ids, attention_masks, token_type_ids])
        loss = F.nll_loss(y_hat, y.view(-1))
        return loss


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-5)

In [39]:
data_loader = TweetDataLoader(train_input, val_input, batch_size=32)
trainer = pl.Trainer(max_epochs=3, gpus=1)
hs_model = HateSpeechClassifier(model)
trainer.fit(hs_model.to(device), data_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type        | Params
----------------------------------------
0 | model   | AlbertModel | 11.7 M
1 | l1      | Linear      | 590 K 
2 | dropout | Dropout     | 0     
3 | l2      | Linear      | 1.5 K 
----------------------------------------
12.3 M    Trainable params
0         Non-trainable params
12.3 M    Total params
49.103    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]